In [1]:
import os, torch
#os.environ["CUDA_VISIBLE_DEVICES"] = "5, 6"

device = "cuda:5"

torch.cuda.set_device(device)

In [2]:
import lighteval, torch

import torch._dynamo
torch._dynamo.config.suppress_errors = True
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from utils.config import Config

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5")

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_config = Config.from_json("assets/sample_config.json")
model_config = AutoConfig.from_pretrained("/common-repos/xLSTM-7b")
model_config.num_blocks = 16
model_config.head_dim = 256
model_config.num_heads = 8
model_config.embedding_dim = 2048
model_config.vocab_size = 51200
student_model = AutoModelForCausalLM.from_pretrained("/common-repos/xLSTM-7b", config=model_config, ignore_mismatched_sizes=True).to(device)

student_model.requires_grad_(True)

Loading checkpoint shards: 100%|██████████| 6/6 [00:01<00:00,  5.81it/s]
Some weights of the model checkpoint at /common-repos/xLSTM-7b were not used when initializing xLSTMForCausalLM: ['backbone.blocks.16.ffn.proj_down.weight', 'backbone.blocks.16.ffn.proj_up.weight', 'backbone.blocks.16.ffn.proj_up_gate.weight', 'backbone.blocks.16.mlstm_layer.fgate_preact.bias', 'backbone.blocks.16.mlstm_layer.fgate_preact.weight', 'backbone.blocks.16.mlstm_layer.igate_preact.bias', 'backbone.blocks.16.mlstm_layer.igate_preact.weight', 'backbone.blocks.16.mlstm_layer.k.weight', 'backbone.blocks.16.mlstm_layer.multihead_norm.weight', 'backbone.blocks.16.mlstm_layer.ogate_preact.weight', 'backbone.blocks.16.mlstm_layer.out_proj.weight', 'backbone.blocks.16.mlstm_layer.q.weight', 'backbone.blocks.16.mlstm_layer.v.weight', 'backbone.blocks.16.norm_ffn.weight', 'backbone.blocks.16.norm_mlstm.weight', 'backbone.blocks.17.ffn.proj_down.weight', 'backbone.blocks.17.ffn.proj_up.weight', 'backbone.blocks.17.

xLSTMForCausalLM(
  (backbone): xLSTMModel(
    (embeddings): Embedding(51200, 2048)
    (blocks): ModuleList(
      (0-15): 16 x mLSTMBlock(
        (norm_mlstm): RMSNorm()
        (mlstm_layer): mLSTMLayer(
          (q): Linear(in_features=2048, out_features=1024, bias=False)
          (k): Linear(in_features=2048, out_features=1024, bias=False)
          (v): Linear(in_features=2048, out_features=2048, bias=False)
          (ogate_preact): Linear(in_features=2048, out_features=2048, bias=False)
          (igate_preact): Linear(in_features=2048, out_features=8, bias=True)
          (fgate_preact): Linear(in_features=2048, out_features=8, bias=True)
          (ogate_act_fn): Sigmoid()
          (mlstm_backend): mLSTMBackend(mLSTMBackendConfig(chunkwise_kernel='chunkwise--triton_xl_chunk', sequence_kernel='native_sequence__triton', step_kernel='triton', mode='inference', chunk_size=64, return_last_states=True, autocast_kernel_dtype='bfloat16', eps=1e-06, inference_state_dtype='float32

In [4]:
path = "checkpoints_logits/epoch_1idx_7104.pt"
checkpoint = torch.load(path, map_location=device, weights_only=True)
model_state_to_load = checkpoint['model_state_dict']


student_model.load_state_dict(model_state_to_load)
student_model.requires_grad_(True)

xLSTMForCausalLM(
  (backbone): xLSTMModel(
    (embeddings): Embedding(51200, 2048)
    (blocks): ModuleList(
      (0-15): 16 x mLSTMBlock(
        (norm_mlstm): RMSNorm()
        (mlstm_layer): mLSTMLayer(
          (q): Linear(in_features=2048, out_features=1024, bias=False)
          (k): Linear(in_features=2048, out_features=1024, bias=False)
          (v): Linear(in_features=2048, out_features=2048, bias=False)
          (ogate_preact): Linear(in_features=2048, out_features=2048, bias=False)
          (igate_preact): Linear(in_features=2048, out_features=8, bias=True)
          (fgate_preact): Linear(in_features=2048, out_features=8, bias=True)
          (ogate_act_fn): Sigmoid()
          (mlstm_backend): mLSTMBackend(mLSTMBackendConfig(chunkwise_kernel='chunkwise--triton_xl_chunk', sequence_kernel='native_sequence__triton', step_kernel='triton', mode='inference', chunk_size=64, return_last_states=True, autocast_kernel_dtype='bfloat16', eps=1e-06, inference_state_dtype='float32

[2025-05-15 20:21:05,203] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


df: /root/.triton/autotune: No such file or directory
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [ ]:
!lighteval accelerate --model_args  "student_model,dtype=bfloat16" --tasks "leaderboard|hellaswag|10|0" --output_dir ./outputs --override_batch_size 32
